Data(Exercise 1a)- Which author has the most 'wrong scope' decisions

In [64]:
#Read the 3 tables into 3 data frames

df1=pd.read_csv('fct_submission.csv')
df2=pd.read_csv('Dim_decision.csv')
df3=pd.read_csv('Dim_autho.csv')

#filter df1 such that we only pick entries where Decision id =2(rejected)
rej_submission=df1[df1['decision_id']==2]
rej_submission

,submission_id,submission_type,author_id,decision_id
0,1,submission,1,2
2,3,submission,3,2
4,5,submission,1,2


In [65]:
#For the rejected submissions merge with author table to gather the names of the author
rej_submission=rej_submission.merge(df3,on='author_id') 
rej_submission

,submission_id,submission_type,author_id,decision_id,author_name,author_institution
0,1,submission,1,2,Maria,University A
1,5,submission,1,2,Maria,University A
2,3,submission,3,2,Mohammed,University A


In [ ]:
#On the merged table roll up the table such that each row contains the author name and number of rejected submissions generated
df_final=rej_submission.groupby(by='author_name').count().reset_index()


In [ ]:

#retain only the necessary columns
df_final_rejected_submissions=df_final[['author_name','submission_id']]

#output dataframe
df_final_rejected_submissions

Data(Exercise 1b)- What is the editor acceptance rate per institution? 

In [41]:
#Already the 3 files have been read

#Merge the Submission table with author table
auth_submission=df3.merge(df1,on='author_id') 

In [45]:
auth_submission

,author_id,author_name,author_institution,submission_id,submission_type,decision_id
0,1,Maria,University A,1,submission,0
1,1,Maria,University A,2,article_transfer,1
2,1,Maria,University A,5,submission,0
3,3,Mohammed,University A,3,submission,0
4,3,Mohammed,University A,3,submission,1
5,4,Jose,University C,4,submission,1


In [44]:
#create a flag column to show acceptance decision id
auth_submission.loc[auth_submission['decision_id'] == 1, 'decision_id'] = 1
auth_submission.loc[auth_submission['decision_id'] != 1, 'decision_id'] = 0

In [57]:
# create a dataframe to show total accepted entries for that university
univ_sum=auth_submission.groupby(by='author_institution').sum().reset_index()
univ_sum=univ_sum[['author_institution','decision_id']]
univ_sum

C:\Users\24057\AppData\Local\Temp\ipykernel_19932\402639935.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  univ_sum=auth_submission.groupby(by='author_institution').sum().reset_index()


,author_institution,decision_id
0,University A,2
1,University C,1


In [56]:
# create a dataframe to show total accepted entries for that university
univ_count=auth_submission.groupby(by='author_institution').count().reset_index()
univ_count=univ_count[['author_institution','submission_id']]
univ_count

,author_institution,submission_id
0,University A,5
1,University C,1


In [62]:
# merge these two dataframes on author_institution
df_final=univ_sum.merge(univ_count,on='author_institution')
#create an acceptance rate column(sum of decision id/count of submission id)
df_final['AR']=df_final['decision_id']/df_final['submission_id']
df_final

,author_institution,decision_id,submission_id,AR
0,University A,2,5,0.4
1,University C,1,1,1.0


Modelling(exercise 3)

In [5]:
import pandas as pd
from scipy import stats

# Load data from a CSV file into a Pandas DataFrame
df = pd.read_csv('Data exercise.csv')

# Filter the data by control and treatment groups
control_data = df[df["control_flag"] == "Control"]["editor_decision_time_seconds"]
treatment_data = df[df["control_flag"] == "Treatment"]["editor_decision_time_seconds"]

# Perform the t-test
t_stat, p_value = stats.ttest_ind(control_data, treatment_data, equal_var=False)

# Print the results
print("t-statistic: ", t_stat)
print("p-value: ", p_value)

t-statistic:  35.042486557233985
p-value:  7.50396756104829e-250


There is a statistically significant difference in editor decision times between the control and treatment groups. The t-statistic of 35.04 indicates that the mean decision times for the two groups are significantly different, and the p-value of 7.50e-250 is much smaller than the conventional alpha level of 0.05, suggesting strong evidence against the null hypothesis that there is no difference in decision times between the control and treatment groups.

In other words, the p-value is less than the chosen significance level of 0.05, indicating that the probability of observing such a large difference in means by chance alone is very low. Therefore, you can reject the null hypothesis and conclude that there is a statistically significant difference in editor decision times between the control and treatment groups.

In [2]:
import pandas as pd
from scipy.stats import chi2_contingency

# Load data from a CSV file into a Pandas DataFrame
df = pd.read_csv('Data exercise.csv')

# Create a contingency table of decision outcomes by control/treatment groups
contingency_table = pd.crosstab(df["control_flag"], df["decision"])

# Perform the chi-squared test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-squared statistic: ", chi2_stat)
print("p-value: ", p_value)
print("Degrees of freedom: ", dof)
print("Expected values: \n", expected)

Chi-squared statistic:  30.035470014785542
p-value:  4.2421525159847305e-08
Degrees of freedom:  1
Expected values: 
 [[2379.12631222 2060.87368778]
 [2367.87368778 2051.12631222]]


The chi-squared statistic of 30.04 indicates that there is a large difference between the observed frequencies and the frequencies expected under the null hypothesis of independence (i.e., no association between the control/treatment group and the decision outcomes). The degrees of freedom is 1, which means there is only one degree of freedom in the chi-squared distribution.

The p-value of 4.24e-08 is less than the conventional alpha level of 0.05, which indicates that the observed association between the control/treatment group and the decision outcomes is statistically significant. Therefore, you can reject the null hypothesis of independence and conclude that there is a significant association between the control/treatment group and the decision outcomes.

In [4]:
pip install statsmodels
import pandas as pd
from statsmodels.tsa.api import ExponentialSmoothing

# load the data as a time series
data = pd.read_csv('Data exercise_2.csv', parse_dates=['Month'], index_col='Month')

# perform exponential smoothing with additive trend and seasonal components
model = ExponentialSmoothing(data, trend='add', seasonal='add', seasonal_periods=12)
fit = model.fit()

# generate forecast for the next 12 periods
forecast = fit.forecast(12)

# print the forecasted values
print(forecast)

SyntaxError: invalid syntax (430716874.py, line 1)

In [ ]:
!pip install statsmodels